# ROB2004 Final Project - pick and place objects

The goal of this project is to solve a simple manipulation task: picking up objects and moving them in a bowl.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a typeset answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Question

Implement a controller using the code skeleton below in order to get the robot to go and pick up the red blocks and drop them in the green bowl. The bowl position is $(-0.3,0.55,0.65)$ (in spatial frame coordinates) and the blocks positions are $(0.35,0.58,0.65)$ and $(0.15,0.67,0.65)$ respectively.

<div>
<img src="./manipulation_scene.jpg" width="300"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to generate smooth motions
* You need to compensate for the gravity of the robot

In your report, describe the controller and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently implement a simple PD controller in joint space that just keeps a desired current position.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

Implement a controller using the code skeleton below in order to get the robot to go and pick up the red blocks and drop them in the green bowl. The bowl position is $(-0.3,0.55,0.65)$ (in spatial frame coordinates) and the blocks positions are $(0.35,0.58,0.65)$ and $(0.15,0.67,0.65)$ respectively.

In [1]:
import numpy as np
import time

import matplotlib.pyplot as plt

from panda import Simulator, PandaRobot, inverseT

np.set_printoptions(precision=4, suppress = True)

# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])

# we create a robot object so we can use its helper functions
my_robot = PandaRobot()

# duration of the simulation
run_time = 40
step = run_time // 4

# simulation time step
dt = 0.005
num_steps = int(run_time/dt)

# we store information
ndofs = 7 # number of degrees of freedom (excluding the gripper)
measured_positions = np.zeros([num_steps,ndofs])
measured_velocities = np.zeros_like(measured_positions)
desired_torques = np.zeros_like(measured_positions)

measured_positions_cart = np.zeros((num_steps, 3))
desired_positions = np.zeros_like(measured_positions_cart)
desired_velocities = np.zeros_like(measured_positions_cart)

time = np.zeros([num_steps])

def compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    T = t_goal - t_init
    a = t-t_init
    
    desired_position = position_init + ((10/T**3)*a**3 + (-15/T**4)*a**4 + (6/T**5)*a**5)*(position_goal-position_init)
    desired_velocity = ((30/T**3)*a**2 + (-60/T**4)*a**3 + (30/T**5)*a**4)*(position_goal-position_init)

    return desired_position, desired_velocity

def stack(pt):
    return np.vstack((pt, np.array([[1]])))

bowl_position = np.array([[-0.3],[0.55],[0.65]])
block_positions = [np.array([[0.37],[0.58],[0.3]]),np.array([[0.15],[0.67],[0.33]])]

### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    measured_positions[i,:] = q
    measured_velocities[i,:] = dq

    # Homogenous transforms 
    T_SE = my_robot.FK(q)
    T_SO = np.copy(T_SE)
    T_SO[0:3, 0:3] = np.eye(3)
    T_OS = inverseT(T_SO)
    T_ES = inverseT(T_SE) 

    # PD gains tuned so the robot stops swaying
    P = np.eye(3) * 15
    D = np.eye(7) * 5

    # Morse-Penrose inverse since J is not invertible 
    Jv = my_robot.get_jacobian(q, 'O')[3:,]
    Jv_inv = Jv.T @ np.linalg.inv(Jv @ Jv.T)

    
    if time[i] <= step*1:  # To Block 1
        x_init = my_robot.FK(measured_positions[0])[:3,3].reshape((3,1))  # Initial position of the end effector in the spacial frame
        x_goal = block_positions[0]  # Block 1 position in the spacial frame
        t_init = 0
        t_goal = step*1
    elif step*1 < time[i] <= step*2:  # To Bowl
        x_init = block_positions[0] 
        x_goal = bowl_position
        t_init = step*1 + dt
        t_goal = step*2
    elif step*2 < time[i] <= step*3:  # To Block 2
        x_init = bowl_position 
        x_goal = block_positions[1]
        t_init = step*2 + dt
        t_goal = step*3
    else:  # To Bowl
        x_init = block_positions[1] 
        x_goal = bowl_position 
        t_init = step*3 + dt
        t_goal = step*4

    x_mes = my_robot.FK(q)[:3,3].reshape((3,1))  # Measured position of the end effector in the spacial frame
    x_ref, v_ref = compute_trajectory(x_init, x_goal, t_init, t_goal, time[i])

    # N = np.eye(7) - Jv_inv @ Jv
    # N_task = ...

    error = x_ref - x_mes
    dq_des = Jv_inv @ (v_ref + P @ error)
    # dq_des = dq_des + N_task
    dq = dq.reshape((7,1))

    g = my_robot.g(q).reshape((7,1))

    joint_torques = D @ (dq_des - dq) + g
 
    gripper_position = [0.00,0.00] # Gripper Closed
    if 0 < time[i] <= step*1 or step*2 < time[i] <= step*3 or time[i] > (step*4 - 50*dt) :
        gripper_position = [0.04, 0.04] # Gripper Open

    measured_positions_cart[i,:] = my_robot.FK(measured_positions[i])[:3,3]
    desired_positions[i,:] = x_ref.reshape(1,3)
    desired_velocities[i,:] = v_ref.reshape(1,3)

    desired_torques[i,:] = joint_torques.reshape(1,7)
    # we send them to the robot and do one simulation step
    simulator.send_joint_torque(joint_torques)
    simulator.gripper_move(gripper_position)
    simulator.step()

ValueError: could not broadcast input array from shape (7,1) into shape (7,)

In [ ]:
def plot_finger_trajectory(finger_position):
    """
    plots the position of the finger in 2D and the position of the spatial frame {s}
    we assume that the time varying x variable is in x_pos and that the y variable is in y_pos
    """
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    plt.plot(finger_position[:,0],finger_position[:,1], 'b')
    plt.xlabel('finger x position [m]')
    plt.ylabel('finger y position [m]')
    plt.plot([0],[0],'o',markersize=15,color='r')
    
    ax.annotate('Spatial frame {s}', xy=(-0.03,-0.005), xytext=(30,0), textcoords='offset points')
    

plot_finger_trajectory(measured_positions_cart)